In [1]:
import yaml
import src.sign_off_cli as sign_off_cli
import src.retrieve_logs as retrieve_logs
import src.twingate_status as twingate_status
from datetime import datetime

Twingate is running.


In [ ]:
benchmark_session_ids = []

In [ ]:
Fixed Set Up Parameters

In [2]:
jenkins_url = "http://jenkins-ber.reeinfra.net/view/Main%20Testbed%20Queues/"
job_name = "job/release-testbed-validation-gamma"
build_info_url = f"{jenkins_url}{job_name}"

In [3]:
if not twingate_status.is_twingate_running():
    print("Twingate is not running. Please start it and try again.")
    exit(1)

credentials = sign_off_cli.get_credentials()
last_successful_build = retrieve_logs.get_last_build_information("Successful", credentials, build_info_url)
console_log = retrieve_logs.get_console_logs(credentials, last_successful_build, build_info_url)
list_of_builds = sign_off_cli.get_last_3_succesfull_builds(last_successful_build, credentials, build_info_url)
metrics_yaml = sign_off_cli.get_log_yaml(console_log)

In [7]:
list_of_builds[0]

{'build': 2518,
 'session_id': '01HC9YHHE08XX54RRHY103HK8Y',
 'branch': 'rc49.0.1',
 'time': 1696845962083}

In [5]:
import requests
console_log = requests.get(f"{build_info_url}/{last_successful_build}/api/json", auth=(credentials["username"],credentials["credentials"]))

In [44]:
console_log.json()['timestamp']

1696845962083

In [52]:
from datetime import datetime
datetime.fromtimestamp((console_log.json()['timestamp'])//1000).strftime('%Y-%m-%d')

'2023-10-09'

In [27]:
all_metrics = sign_off_cli.collect_metrics(list_of_builds, credentials, build_info_url)

In [28]:
metrics_average = sign_off_cli.get_metrics_average(all_metrics)

In [29]:
front_camera_packets = 'front camera packets per second_packets_per_second'

list_of_keys_camera_bitrate = ['front camera bitrate_kbits_per_second', 
                'left camera bitrate_kbits_per_second',
                'right camera bitrate_kbits_per_second',
                'back camera bitrate_kbits_per_second']

list_of_keys_camera_new_unit = ['front camera Mbit/s',
                              'left camera Mbit/s',
                              'right camera Mbit/s',
                              'back camera Mbit/s']

list_of_keys_udp_tcp_bitrate = ['ROS UDP_kbits_per_second',
                'ROS TCP_kbits_per_second',
                'UDP TS-VE_kbits_per_second',
                'UDP VE-TS_kbits_per_second',
                'TCP TS-VE_kbits_per_second',
                'TCP VE-TS_kbits_per_second']

list_of_keys_udp_tcp_new_unit = ['ROS UDP Mbit/s', 
                         'ROS TCP Mbit/s',
                         'UDP TS-VE Mbit/s',
                         'UDP VE-TS Mbit/s',
                         'TCP TS-VE Mbit/s',
                         'TCP VE-TS Mbit/s']
print('Average of following builds:', [build['build'] for build in list_of_builds], "on Branch:", list_of_builds[0]["branch"])
print('front camera packets/s','\t\t',
      round(metrics_average[front_camera_packets]['minimum']['value']), 
      round(metrics_average[front_camera_packets]['median']['value']),
      round(metrics_average[front_camera_packets]['max']['value']))

for key in zip(list_of_keys_camera_bitrate, list_of_keys_camera_new_unit):
      if len(key[1]) < 16:
            tab = '\t\t\t'
      else:
            tab = '\t\t'
      print(key[1],tab,
          round(metrics_average[key[0]]['minimum']['value']/1000,2), 
          round(metrics_average[key[0]]['median']['value']/1000,2),
          round(metrics_average[key[0]]['max']['value']/1000,2))
print("----------------------------------------")
print("UDP/TCP Bitrate")
for key in zip(list_of_keys_udp_tcp_bitrate, list_of_keys_udp_tcp_new_unit):

      if len(key[1]) < 16:
            tab = '\t\t\t'
      else:
            tab = '\t\t'

      print(key[1],tab,
          round(metrics_average[key[0]]['minimum']['value']/1000,2), 
          round(metrics_average[key[0]]['median']['value']/1000,2),
          round(metrics_average[key[0]]['max']['value']/1000,2))

Average of following builds: [2518] on Branch: rc49.0.1
front camera packets/s 		 325 333 339
front camera Mbit/s 		 3.35 3.45 3.5
left camera Mbit/s 		 1.5 1.52 1.55
right camera Mbit/s 		 1.49 1.52 1.55
back camera Mbit/s 		 0.95 0.97 1.0
----------------------------------------
UDP/TCP Bitrate
ROS UDP Mbit/s 			 0.05 0.05 0.05
ROS TCP Mbit/s 			 0.02 0.03 0.03
UDP TS-VE Mbit/s 		 0.02 0.02 0.02
UDP VE-TS Mbit/s 		 0.03 0.03 0.03
TCP TS-VE Mbit/s 		 0.01 0.01 0.01
TCP VE-TS Mbit/s 		 0.02 0.02 0.02


In [30]:
print("test bed session IDs for redash:")
print("----------------------------------------")
for build in list_of_builds:
    print(build['session_id'])

test bed session IDs for redash:
----------------------------------------
01HC9YHHE08XX54RRHY103HK8Y


In [15]:

p_session_id = [0,0,0,0,0,0,0,0]

for i in range(min(len(list_of_builds), len(p_session_id))):
    p_session_id[i] = list_of_builds[i]['session_id']

start_date = datetime.fromtimestamp(list_of_builds[0]['time']//1000).strftime('%Y-%m-%d')
end_date = datetime.fromtimestamp(list_of_builds[-1]['time']//1000).strftime('%Y-%m-%d')
release_candidate = list_of_builds[0]['branch']


print("Testbed: release testing metrics - v2")
print(f"https://redash.reeinfra.net/dashboards/96-testbed-release-testing-metrics---v2?p_date={start_date}--{end_date}&p_date.end={end_date}&p_date.start={start_date}&p_date_1=0&p_date_2=0&p_env=prod&p_session_id_1={p_session_id[0]}&p_session_id_2={p_session_id[1]}&p_session_id_3={p_session_id[2]}&p_session_id_4={p_session_id[3]}&p_session_id_5={p_session_id[4]}&p_session_id_6={p_session_id[5]}&p_session_id_7={p_session_id[6]}&p_session_id_8={p_session_id[7]}&p_subsample_percent=5&p_vdrive_version={release_candidate}&refresh=600")

Testbed: release testing metrics - v2
https://redash.reeinfra.net/dashboards/96-testbed-release-testing-metrics---v2?p_date=2023-10-09--2023-10-09&p_date.end=2023-10-09&p_date.start=2023-10-09&p_date_1=0&p_date_2=0&p_env=prod&p_session_id_1=01HC9YHHE08XX54RRHY103HK8Y&p_session_id_2=0&p_session_id_3=0&p_session_id_4=0&p_session_id_5=0&p_session_id_6=0&p_session_id_7=0&p_session_id_8=0&p_subsample_percent=5&p_vdrive_version=rc49.0.1&refresh=600


In [10]:
p_session_id = [0,0,0,0,0,0,0,0]

for i in range(min(len(benchmark_session_ids = []), len(p_session_id))):
    p_session_id[i] = benchmark_session_ids[i]
print("Release Testing / Production Metrics")
print(f"https://redash.reeinfra.net/dashboards/61-release-testing-production-metrics?p_env=prod&p_session_id_1={p_session_id[0]}&p_session_id_2={p_session_id[1]}&p_session_id_3={p_session_id[2]}&p_session_id_4={p_session_id[3]}&p_session_id_5={p_session_id[4]}&p_session_id_6={p_session_id[5]}&p_session_id_7={p_session_id[6]}&p_session_id_8={p_session_id[7]}&p_w1652_subsample_percent=5&p_w1653_subsample_percent=5&p_w1656_subsample_percent=5")

'2023-10-09'